Import packages

In [1]:
import pandas as pd
import numpy as np  
import re #for datacleaning
import nltk  
nltk.download('stopwords')  
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import accuracy_score #for measuring accuracy in text category models
from sklearn.model_selection import train_test_split #to divide in train and test set
from sklearn.feature_extraction.text import TfidfVectorizer  #for tfidf vectorize
from sklearn import naive_bayes, svm #for text categorization models
from sklearn.feature_extraction.text import CountVectorizer #for vectorizing
from sklearn.decomposition import LatentDirichletAllocation #for the topic mining model

[nltk_data] Downloading package stopwords to /Users/Amy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


The packages <b> langdetect </b> and <b> pyLDAvis </b> need to be downloaded and installed using pip. 

In [2]:
from langdetect import detect
import pyLDAvis.sklearn

# Text Categorization

Import datasets. The datasets are webscraped from the lists on Goodreads: <br>
for non-fiction: https://www.goodreads.com/list/show/134.Best_Non_Fiction_no_biographies_ <br>
for fiction: https://www.goodreads.com/list/show/2522.Cult_Classics

In [3]:
nf = pd.read_csv('booksSummaryNF.txt', sep="|", header=None)
f = pd.read_csv("booksSummaryCult_Classics.txt",sep="|", header=None)

I rename datasets' columns and add a new column specifing the genre original dataset

In [4]:
f = f.rename(columns={0: "title", 1: "rating",2:"summary"})
nf = nf.rename(columns={0: "title", 1: "rating",2:"summary"})
f["genre"]="fiction"
nf["genre"]="non fiction"

I append the two datasets.

In [5]:
df=f.append(nf,ignore_index = True)

I define the genre of the book (fiction or non-fiction) as the target value. 

In [6]:
X=df["summary"]
y=df["genre"]

Perform data cleaning on the dataset.

In [8]:
documents = []
stemmer = WordNetLemmatizer()

for i in range(0, len(X)):  
    document = re.sub(r'\W', ' ', str(X[i])) #remove all the special characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document) #remove all single characters
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) #remove single characters from the start
    document = re.sub(r'\s+', ' ', document, flags=re.I)  # substitute multiple spaces with single space
    document = document.lower() #converte to Lowercase
    document = document.split() #lemmatization
    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    documents.append(document)

I divide the dataset in test and train.

In [9]:
Train_X, Test_X, Train_Y, Test_Y = train_test_split(documents,y,test_size=0.3)

I apply TfidfVectorizer

In [10]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(documents)
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

<b> Naive Bayes classifier</b> model, first fit the model on the train set then predict the values of the test set and calculate the accuracy:

In [11]:
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)
predictions_NB = Naive.predict(Test_X_Tfidf)
print("Naive Bayes Accuracy Score -> ",accuracy_score(y_pred=predictions_NB, y_true=Test_Y)*100)

Naive Bayes Accuracy Score ->  84.95726495726495


<b> SVM</b> model, first fit the model on the train set then predict the values of the test set and calculate the accuracy:

In [12]:
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
predictions_SVM = SVM.predict(Test_X_Tfidf)
print("SVM Accuracy Score -> ",accuracy_score(y_pred=predictions_SVM,y_true= Test_Y)*100)

SVM Accuracy Score ->  84.61538461538461


# Topic Mining

Import datasets. The datasets are webscraped from the lists on Goodreads: <br>
for non-fiction: https://www.goodreads.com/list/show/134.Best_Non_Fiction_no_biographies_ <br>
for fiction: https://www.goodreads.com/list/show/2522.Cult_Classics <br>
for young adult: https://www.goodreads.com/list/show/43.Best_Young_Adult_Books <br>
for sci-fi: https://www.goodreads.com/list/show/19341.Best_Science_Fiction <br>
for fantasy: https://www.goodreads.com/list/show/50.The_Best_Epic_Fantasy_fiction_ <br>
for thriller: https://www.goodreads.com/list/show/348.Thrillers <br>
for romance: https://www.goodreads.com/list/show/12362.All_Time_Favorite_Romance_Novels

In [13]:
ya= pd.read_csv("booksSummaryYoung_Adult.txt", sep="|",header=None)
f = pd.read_csv("booksSummaryCult_Classics.txt", sep="|", header=None)
nf = pd.read_csv('booksSummaryNF.txt', sep="|", header=None)
rom=pd.read_csv("booksSummaryRomance_Novels.txt", sep="|", header=None)
thr=pd.read_csv("booksSummaryThrillers.txt", sep="|", header=None)
sifi=pd.read_csv("booksSummaryScience_Fiction.txt", sep="|", header=None)
fan=pd.read_csv("booksSummaryEpic_Fantasy.txt", sep="|", header=None)
f = f.rename(columns={0: "title", 1: "rating",2:"summary"})
ya = ya.rename(columns={0: "title", 1: "rating",2:"summary"})
nf = nf.rename(columns={0: "title", 1: "rating",2:"summary"})
rom = rom.rename(columns={0: "title", 1: "rating",2:"summary"})
thr = thr.rename(columns={0: "title", 1: "rating",2:"summary"})
sifi = sifi.rename(columns={0: "title", 1: "rating",2:"summary"})
fan=fan.drop(columns=[0])
fan = fan.rename(columns={1: "title", 2: "rating", 3:"summary"})

I unite the datasets.

In [14]:
fic=f.append(ya,ignore_index = True)
fic=fic.append(nf,ignore_index=True)
fic=fic.append(rom,ignore_index=True)
fic=fic.append(thr,ignore_index=True)
fic=fic.append(sifi,ignore_index=True)
fic=fic.append(fan,ignore_index=True)

For <b>Data Cleaning</b>, I detect the language of each summary using library langdetect

In [15]:
fic["language"]=""
for i in range(0,len(fic)):
    fic.at[i,"language"]=detect(fic.loc[i,"summary"])

I drop the duplicated titles.

In [16]:
fic_clean=fic.drop_duplicates(['title'])

I take only the summaries in English.

In [17]:
fic_final=fic_clean.loc[fic_clean['language'] == "en"]
print("The dataset has ", fic_final.shape[0], "summaries.")

The dataset has  5700 summaries.


I create a vector for the words in the data set and a LDA model, with 6 topics.

In [18]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)
dtm_tf = tf_vectorizer.fit_transform(fic_final["summary"])
print(dtm_tf.shape)
lda_tf = LatentDirichletAllocation(n_components=6, random_state=0)
lda_tf.fit(dtm_tf)

(5700, 6530)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=6, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

Visualizing the topics

In [19]:
pyLDAvis.enable_notebook()

In [20]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

/Users/Amy/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.026798  0.195927       1        1  27.265409
4     -0.008706 -0.102234       2        1  19.848352
5      0.038630  0.034717       3        1  17.884263
1      0.201336 -0.028605       4        1  17.522432
3     -0.051398 -0.063917       5        1  13.012540
0     -0.153065 -0.035888       6        1   4.467003, topic_info=     Category         Freq     Term        Total  loglift  logprob
3463  Default  1533.000000     love  1533.000000  30.0000  30.0000
2492  Default   535.000000     girl   535.000000  29.0000  29.0000
3252  Default   490.000000     king   490.000000  28.0000  28.0000
6320  Default  1110.000000      war  1110.000000  27.0000  27.0000
3515  Default   524.000000    magic   524.000000  26.0000  26.0000
218   Default   473.000000  ancient   473.000000  25.0000  25.0000
3380  Default  2635.000000     life  2635.000000  24.0000  24.0000
6513  Default   755.000000     year   755.000000  23.0000  23.0000
3211  Default   963.000000     just   963.000000  22.0000  22.0000
2821  Default   902.000000    human   902.000000  21.0000  21.0000
2757  Default   733.000000  history   733.000000  20.0000  20.0000
3389  Default   968.000000     like   968.000000  19.0000  19.0000
1989  Default   415.000000     evil   415.000000  18.0000  18.0000
3415  Default   472.000000   little   472.000000  17.0000  17.0000
4243  Default   387.000000   planet   387.000000  16.0000  16.0000
3984  Default   883.000000      old   883.000000  15.0000  15.0000
1755  Default   694.000000    earth   694.000000  14.0000  14.0000
3285  Default   382.000000     land   382.000000  13.0000  13.0000
6480  Default  2913.000000    world  2913.000000  12.0000  12.0000
1393  Default   625.000000      day   625.000000  11.0000  11.0000
481   Default   372.000000   battle   372.000000  10.0000  10.0000
3241  Default   377.000000   killer   377.000000   9.0000   9.0000
634   Default  1586.000000     book  1586.000000   8.0000   8.0000
654   Default   422.000000      boy   422.000000   7.0000   7.0000
1654  Default   367.000000    doesn   367.000000   6.0000   6.0000
6459  Default   797.000000    woman   797.000000   5.0000   5.0000
4341  Default   810.000000    power   810.000000   4.0000   4.0000
167   Default   238.000000    alien   238.000000   3.0000   3.0000
1374  Default   704.000000     dark   704.000000   2.0000   2.0000
3879  Default  2339.000000      new  2339.000000   1.0000   1.0000
...       ...          ...      ...          ...      ...      ...
2238   Topic6    40.212918     film    98.134993   2.2163  -5.9981
3812   Topic6    21.897771   museum    35.768412   2.6178  -6.6059
569    Topic6    20.775798    billy    32.708981   2.6546  -6.6585
6513   Topic6   104.017036     year   755.463478   1.1257  -5.0478
4995   Topic6    35.766730      sam    95.382233   2.1276  -6.1153
2560   Topic6    36.909193    green   107.267260   2.0416  -6.0839
6458   Topic6    19.841907   wolves    30.937214   2.6643  -6.7045
3415   Topic6    73.415420   little   472.683170   1.2462  -5.3962
1393   Topic6    82.431437      day   625.888075   1.0812  -5.2803
23     Topic6    30.169713  academy    75.935473   2.1854  -6.2855
4234   Topic6    70.690161    place   507.260884   1.1377  -5.4340
6470   Topic6    30.512294    woods    78.605186   2.1621  -6.2742
3211   Topic6    89.458538     just   963.572097   0.7316  -5.1985
654    Topic6    61.909856      boy   422.808680   1.1872  -5.5666
2821   Topic6    80.321800    human   902.089475   0.6898  -5.3063
1654   Topic6    54.947062    doesn   367.192618   1.2089  -5.6859
3879   Topic6   103.242020      new  2339.144792  -0.0120  -5.0552
3984   Topic6    71.014272      old   883.938000   0.5869  -5.4294
546    Topic6    59.743235     best   708.089153   0.6359  -5.6023
6523   Topic6    64.194987    young   994.634440   0.3680  -5.5304
657    Topic6    34.688293     boys   130